# EBEOBsys

This document explains the architecture and training of U-net type ANN for recognition of earthworms in terrariums.

To install the required packages use the following line:

If using conda:

```bash
conda isntall tensorflow, numpy
```

otherwise: 

```bash
pip install tensorflow, numpy
```



## Importing the required packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import (Conv2D, Conv2DTranspose, Input,
                                     Lambda, MaxPooling2D, concatenate)
from tensorflow.keras.models import Model


We preprocessed the training an validation images and stored them in binary form using `numpy`.
The dataset contained more than 10gb of data.

In [ ]:
img_width = 816
img_height = 608
img_channels = 3

train = np.load('data/train.npy', mmap_mode='r')
targets = np.load('data/targets_train.npy', mmap_mode='r')
test = np.load('data/test.npy', mmap_mode='r')
test_targets = np.load('data/targets_test.npy', mmap_mode='r')
print (train.shape)

# U-net

U-net is convolutional neural network that uses upsampling factors to propagate information from input to output withous loss of resolution. The network consists of contracting part and epxansive part, hence the U-shape architecture. Contracting part is a standard convolutional network with repeated convlutions and ReLu and max pooling operations. Expansive part contains up-convolutions and concatenations with high-resolution features from the contracting path.

![https://ars.els-cdn.com/content/image/1-s2.0-S001670611931496X-gr3.jpg](https://ars.els-cdn.com/content/image/1-s2.0-S001670611931496X-gr3.jpg)



Build the architecture

In [ ]:
inputs = Input((img_height, img_width, img_channels))
s = Lambda(lambda x: x/255) (inputs)

# Contracting path
c1 = Conv2D(filters=8, kernel_size=(3,3), padding='same', activation='relu') (s)
c1 = Conv2D(filters=8, kernel_size=(3,3), padding='same', activation='relu') (c1)
p1 = MaxPooling2D(pool_size=(2,2)) (c1)

c2 = Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu') (p1)
c2 = Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu') (c2)
p2 = MaxPooling2D(pool_size=(2,2)) (c2)

c3 = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu') (p2)
c3 = Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu') (c3)
p3 = MaxPooling2D(pool_size=(2,2)) (c3)

c4 = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu') (p3)
c4 = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu') (c4)
p4 = MaxPooling2D(pool_size=(2,2)) (c4)

c5 = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu') (p4)
c5 = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu') (c5)

# Expansive path
u6 = Conv2DTranspose(filters=64, kernel_size=(2,2), strides=(2,2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(filters=64, kernel_size=(3,3), padding='same') (u6)
c6 = Conv2D(filters=64, kernel_size=(3,3), padding='same') (c6)

u7 = Conv2DTranspose(filters=32, kernel_size=(2,2), strides=(2,2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(filters=32, kernel_size=(3,3), padding='same') (u7)
c7 = Conv2D(filters=32, kernel_size=(3,3), padding='same') (c7)

u8 = Conv2DTranspose(filters=16, kernel_size=(2,2), strides=(2,2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(filters=16, kernel_size=(3,3), padding='same') (u8)
c8 = Conv2D(filters=16, kernel_size=(3,3), padding='same') (c8)

u9 = Conv2DTranspose(filters=8, kernel_size=(2,2), strides=(2,2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(filters=8, kernel_size=(3,3), padding='same') (u9)
c9 = Conv2D(filters=8, kernel_size=(3,3), padding='same') (c9)
outputs = Conv2D(filters=1, kernel_size=(1,1), activation='softmax') (c9)


## Define the model

In [ ]:
model = Model(inputs=[inputs], outputs=[outputs])
opt = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Training

In [ ]:

mname = 'model_2000'
checkpointer = ModelCheckpoint('models/'+mname, verbose=0, save_best_only=True)
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='models/model_2000_graph', histogram_freq=0, write_graph=True, write_images=False)
results = model.fit(train, targets, validation_data=(test, test_targets), batch_size=8, epochs=6000, initial_epoch=0, callbacks=[checkpointer, tbCallBack])
